In [131]:
from pathlib import Path
import pandas as pd
import logging
from tqdm import tqdm

tqdm.pandas()

logger = logging.getLogger()

### Load Model and Tokenizer

In [132]:
from transformers import T5TokenizerFast, T5ForConditionalGeneration

MODEL_CKPT = 'paust/pko-t5-larget'
MODEL_CKPT = 'paust/pko-t5-base'

tokenizer = T5TokenizerFast.from_pretrained(MODEL_CKPT)
# model = T5ForConditionalGeneration.from_pretrained(MODEL_CKPT)

print(MODEL_CKPT)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


paust/pko-t5-base


### Load Data

In [133]:
def _load_json_data(data_file_dir: Path) -> pd.DataFrame:
    if data_file_dir.exists():
        df = pd.read_json(data_file_dir, encoding="utf-8", orient="records", lines=True)
    else:
        logger.info(f"{data_file_dir} is not exists, return empty ")
        df = pd.DataFrame()
    return df

In [134]:
train_data1 = Path("./data/news_topic_trainset1.json")
valid_data1 = Path("./data/news_topic_validset1.json")
train_data2 = Path("./data/news_topic_trainset2.json")
valid_data2 = Path("./data/news_topic_validset2.json")

### v1 and v2 Comparison

In [135]:
# df_v1 = pd.concat([_load_json_data(train_data1), _load_json_data(valid_data1)]).reset_index(drop=True)
# df_v2 = pd.concat([_load_json_data(train_data2), _load_json_data(valid_data2)]).reset_index(drop=True)

# df_v1.rename(columns={"prompt": "input_text", "completion": "target_text"}, inplace=True)
# df_v2.rename(columns={"prompt": "input_text", "completion": "target_text"}, inplace=True)

In [136]:
# df_v1.target_text.progress_apply(lambda x: len(tokenizer(x).input_ids)).describe()

In [137]:
# for idx, row in df_v1.iterrows():
#     # print(row.input_text)
#     print(row.target_text)
#     print()

In [138]:
# text = '전북도,경기도,자동차 대체 인증부품 산업,활성화,조례 제정,협약 체결,순정품 대비 저렴한 제품,생산·유통·소비 지원,제도개선·정책사업 기획,소비 확대 홍보,송하진 지사,이재명 지사,판로 개척,소비자 선택권,공정경제'
# first_ten = ','.join(text.split(',')[:10])
# len(tokenizer.encode(first_ten))

In [139]:
# df_v2.target_text.progress_apply(lambda x: len(tokenizer(x).input_ids)).describe()

In [140]:
# for idx, row in df_v2.iterrows():
#     # print(row.input_text)
#     print(row.target_text)
#     print()

### Load v2

In [141]:
df_data = pd.concat([_load_json_data(train_data2), _load_json_data(valid_data2)])

In [142]:
for idx, row in df_data.iterrows():
    if idx == 10:
        break
    print(f'Row {idx}')
    print('input')
    print(row.prompt)
    print()
    print('output')
    print(row.completion)
    print()
    print()

Row 0
input
올 가을 단풍 구경 ‘남한의 정중앙’ 배꼽마을 어때요
‘배꼽마을’로 알려진 충북 옥천군 청성면 장연리에는 ‘남한 면적중심 마을’이라는 기념비가 서 있다.
2일 옥천군에 따르면 장연리 마을은 위도 36.34도, 경도는 127.77도로 땅을 기준으로 했을 때 대한민국 가운데 자리 잡고 있다.
2003년 10월 대한지리학회와 국토연구원에서 신행정수도 이전을 위한 조사결과 이곳을 남한 국토의 중앙점이라고 발표했다.
당시 주성재 경희대 교수는 “인구중심점, 면적중심점, 산업중심점 등 국토 중심점이 모두 충청권에 있어 새 행정수도가 이 지역에 들어서는 것이 타당하다”고 주장했다.
면적중심점으로는 옥천군 청성면 장연리, 인구중심점으로는 청원군 가덕면 청룡리, 산업중심점으로는 청원군 남일면 월오리가 명시됐다.
인구를 가중치로 사용한 인구중심점과 산업종사자 수를 가중치로 한 산업중심점은 변할 수 있지만 면적중심점은 바뀌지 않는다.
이런 내용이 알려진 후 이곳은 ‘배꼽마을’로 불린다.
이 마을은 경부고속도로 옥천 나들목을 나와 40여 분을 달려야 도착한다.
내비게이션에 ‘청성면 장연리 산 86-1번지’를 검색해 이동하면 개울 건너편에 어른 키 정도 높이의 돌탑이 보인다.
이곳이 대한민국의 중심 지점이다.
기념비를 지나 300m 정도 안길로 들어서면 23가구가 모여 사는 장명골이 보인다.
여기서 2.5㎞ 정도 더 오르다 보면 저수지를 지나 17가구가 사는 귀재마을에 도착한다.
골짜기가 길게 이어졌다고 해서

###



output
배꼽마을,가을단풍,옥천군,중심점



Row 1
input
NH농협은행, ‘농협이 곧 ESG’, 상생경영 시동건다
환경·사회책임·지배구조를 생각하는 경영이 기업의 새로운 트렌드로 떠올랐다.
기업에 자금을 지원하거나 투자하는 국내 금융사는 ESG경영에 속도를 내며 ESG 리딩금융그룹으로 변신을 시작했다.
단순히 매출과 순이익 등 재무적 요소를 넘어 친환경·사회적 책임 경영·지배구조 개선 등 비재무적 노력을 다해야 금융회사의 지속 성

### Preprocess

In [143]:
def remove_ending_seps(string):
    if string[0] == ',':
        string = string[1:]
    if string[-1] == ',':
        string = string[:-1]
    return string

test_case = ',농협은행,ESG경영,상생경영,제1회 대한민국 리딩금융 ESG 어워드,친환경경영,지배구조,녹색금융,'

remove_ending_seps(test_case)


'농협은행,ESG경영,상생경영,제1회 대한민국 리딩금융 ESG 어워드,친환경경영,지배구조,녹색금융'

In [144]:
def preprocess(df, num_key_phrase):
    df.rename(columns={'prompt': 'input_text', 'completion': 'target_text'}, inplace=True)

    df['input_text'] = df['input_text'].progress_apply(lambda row: row.strip().replace('###', '').strip())
    df['input_text'] = df['input_text'].str.replace('\n', ' ')

    df['target_text'] = df['target_text'].progress_apply(lambda row: ','.join([sentence.strip() for sentence in row.strip().split(',')]))
    df['target_text'] = df['target_text'].progress_apply(remove_ending_seps)
    df['target_text'] = df['target_text'].progress_apply(lambda row: ','.join(row.split(',')[:7]))
    
    return df

In [145]:
preprocessed_data = preprocess(df_data, 7)

100%|██████████| 1585/1585 [00:00<00:00, 599132.29it/s]


In [146]:
for idx, row in cleaned_data.iterrows():
    # if idx == 10:
    #     break
    print(f'Row {idx}')
    # print('input_text')
    # print(row.input_text)
    # print()
    print('target_text')
    print(row.target_text)
    print()
    print()

Row 0
target_text
배꼽마을,가을단풍,옥천군,중심점


Row 1
target_text
농협은행,ESG경영,상생경영,제1회 대한민국 리딩금융 ESG 어워드,친환경경영,지배구조,녹색금융


Row 2
target_text
요소수,국회운영위


Row 3
target_text
노지훈,장모,응급실,살림하는 남자들 시즌2


Row 4
target_text
정의용 외교부 장관,러시아 출국


Row 5
target_text
출산 정책,헝가리,출산정책,인구감소문제


Row 6
target_text
북한은 지금,농촌 살림집 건설


Row 7
target_text
사망전 기관지염,카타르 월드컵,동성애 시위


Row 8
target_text
한미재계회의 환영만찬,김진표 국회의장,허창수 전경련 회장


Row 9
target_text
스토킹 피해,유튜버 릴카


Row 10
target_text
최대 실적,주가 상향,삼성엔지니어링


Row 11
target_text
충북,먹깨비,정체기,대대적인 홍보,돌파구,배달앱,활성화 지원사업


Row 12
target_text
생활가전,역대 최고,LG전자,매출,영업이익,생활가전


Row 13
target_text
중국,손준호,중국슈퍼리그,프리미어리그,유시쯔쉰왕,EPL,리그랭킹 하락


Row 14
target_text
중소기업벤처기업부,중기부,사업연계형 기술개발 지원사업,중소기업


Row 15
target_text
이장호씨,빙부상


Row 16
target_text
맞춤형 콘텐츠,세라젬,리서치,부모님 연구소


Row 17
target_text
LALISA',스우파,리정 지효 이삭,MV 출연,YGX,블랙핑크 리사


Row 18
target_text
티앤엘,결산배당,주당 400원


Row 19
target_text
송파 추월서울,서초구,반포·잠원 아파트,반포·잠원동,강남3구,토지거래허가구역,신고가


Row 20
target_text
한국형 스마트팜 온실 방문,이호승 청와대 정책실장


Row 21
target

### Model Test

In [147]:
# input_ids = tokenizer("qa question: 당신의 이름은 무엇인가요?", return_tensors='pt').input_ids
# labels = tokenizer("T5 입니다.", return_tensors='pt').input_ids

# input_ids, '\n', labels

# outputs = model(input_ids=input_ids, labels=labels)

# print(f"loss={outputs.loss} logits={outputs.logits}")

In [148]:
# output = model.generate(input_ids, max_length=512)
# tokenizer.batch_decode(output, skip_special_tokens=True)

### Sequence Length Check

In [149]:
preprocessed_data.input_text.progress_apply(lambda x: len(tokenizer(x).input_ids)).describe()

100%|██████████| 1585/1585 [00:00<00:00, 2073.87it/s]


count    1585.000000
mean      376.620189
std       172.613232
min         7.000000
25%       215.000000
50%       483.000000
75%       512.000000
max       652.000000
Name: input_text, dtype: float64

In [150]:
preprocessed_data.target_text.progress_apply(lambda x: len(tokenizer(x).input_ids)).describe()

100%|██████████| 1585/1585 [00:00<00:00, 14167.29it/s]


count    1585.000000
mean       21.183596
std         9.877839
min         4.000000
25%        14.000000
50%        19.000000
75%        27.000000
max        74.000000
Name: target_text, dtype: float64

### Test Encoding

In [151]:
prefix = "key phrase generation: "

max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = prefix + examples["input_text"]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")

    labels = tokenizer(examples["target_text"], max_length=max_target_length, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [152]:
model_inputs = preprocess_function(preprocessed_data.iloc[0])

In [153]:
tokenizer.decode(model_inputs['input_ids'])

'key phrase generation: 올 가을 단풍 구경 ‘남한의 정중앙’ 배꼽마을 어때요 ‘배꼽마을’로 알려진 충북 옥천군 청성면 장연리에는 ‘남한 면적중심 마을’이라는 기념비가 서 있다. 2일 옥천군에 따르면 장연리 마을은 위도 36.34도, 경도는 127.77도로 땅을 기준으로 했을 때 대한민국 가운데 자리 잡고 있다. 2003년 10월 대한지리학회와 국토연구원에서 신행정수도 이전을 위한 조사결과 이곳을 남한 국토의 중앙점이라고 발표했다. 당시 주성재 경희대 교수는 “인구중심점, 면적중심점, 산업중심점 등 국토 중심점이 모두 충청권에 있어 새 행정수도가 이 지역에 들어서는 것이 타당하다”고 주장했다. 면적중심점으로는 옥천군 청성면 장연리, 인구중심점으로는 청원군 가덕면 청룡리, 산업중심점으로는 청원군 남일면 월오리가 명시됐다. 인구를 가중치로 사용한 인구중심점과 산업종사자 수를 가중치로 한 산업중심점은 변할 수 있지만 면적중심점은 바뀌지 않는다. 이런 내용이 알려진 후 이곳은 ‘배꼽마을’로 불린다. 이 마을은 경부고속도로 옥천 나들목을 나와 40여 분을 달려야 도착한다. 내비게이션에 ‘청성면 장연리 산 86-1번지’를 검색해 이동하면 개울 건너편에 어른 키 정도 높이의 돌탑이 보인다. 이곳이 대한민국의 중심 지점이다. 기념비를 지나 300m 정도 안길로 들어서면 23가구가 모여 사는 장명골이 보인다. 여기서 2</s>'

In [154]:
tokenizer.decode(model_inputs['labels'])

'배꼽마을,가을단풍,옥천군,중심점</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

### Save Preprocessed Data

In [155]:
# df_train.to_json(
#     "./data/prepared/train_set.json",
#     orient="records",
#     lines=True,
#     force_ascii=False,
# )
# df_valid.to_json(
#     "./data/prepared/valid_set.json",
#     orient="records",
#     lines=True,
#     force_ascii=False,
# )

In [156]:
df_data.to_pickle('./data/preprocess_v2.pickle')

### Check Data

In [157]:
checker = df_data.target_text.apply(lambda x: len(tokenizer.encode(x))) > 128
checked = df_data[checker]

for idx, row in checked.iterrows():
    print(row.target_text)
    print()

opens its doors for the first time in seven years,in Seongbuk District,also known as Bohwagak,Kansong Art Museum,central Seoul

신간 다이제스트,이해하기 쉬운 현대과학 안내서과학은 미래로 흐른다,미래 뒤흔들 4가지 혁신기술앞으로 10년 빅테크 수업,중력이 절실한 청춘에게무중력의 사랑,샤이좌파'에게 보내는 메시지슬기로운 좌파생활



In [158]:
checker = df_data.input_text.str.contains('인천항')
df_data[checker].input_text.iloc[0]

'관세청, 신성장 산업 발전을 위한 관세행정 집중 지원 관세청은 30일, 코로나19 극복을 통한 경제회복 및 신산업 육성을 위해 인천지역 내 바이오, 반도체, 자동차 등 업종의 주요기업과 간담회를 개최했다. 이날 간담회에서 임 청장은 기업 대표들로부터 다양한 건의·어려움을 청취한 뒤 사안별로 관세행정 지원방안에 대해 논의했다. 바이오 기업들은 ▲보세공장 내 원재료 인정 범위 확대, ▲국내 생산 코로나 진단도구의 보세공장 보관 한시적 허용, ▲보세공장 특허 획득을 위한 상담 지원 등을, 반도체 기업들은 ▲시설투자 등에 대한 세제혜택 확대, ▲기 수출된 장비의 사후관리를 위한 부품의 수출요건 완화 등을, 자동차 기업들은 ▲자동차용 반도체 신속통관 지원, ▲자유무역협정 협정세율 적용 심사기간 단축 등을 요청했다. 이 밖에도 ▲인천항만 지역 활성화를 위한 인천세관의 역할 확대와 수출입업체의 지원 요구에 신속히 대응할 수 있는 조직운영, ▲지속적 산업지원 정책 마련·홍보를 건의했다. 이에 임 청장은 “요청사항을 적극적으로 검토해 빠른 시일 내에 해결될 수 있도록 지원하는 한편, 다른 부처와 협력이 필요한 사항에 대해서도 긴밀한 협업을 통해 대책이 수립되도록 노력하겠다”고 답했다. 기업간담회를 마치며 임 청장은“빠르고 강한 경제회복과 함께 신성장 동력산업을 적극 육성하기 위해 관세행정 역량을 집중하겠다”고 밝히면서, 이를 위해, 현 인천세관을 인천공항과 인천항만의 특성에 맞는 차별화된 서비스와 보다'